# Практическое задание 7: Настройка пользовательских правил для NER
В этом задании мы научимся добавлять и настраивать пользовательские правила для **Named Entity Recognition (NER)** с использованием **EntityRuler** в **spaCy**.

Категории именованных сущностей (NER) в spaCy
PERSON: Люди, включая вымышленных персонажей.

NORP: Национальности, религиозные и политические группы.

FAC: Здания, аэропорты, дороги, мосты и другие сооружения.

ORG: Компании, агентства, учреждения и организации.

GPE: Геополитические объекты – страны, города, штаты.

LOC: Локации, которые не являются GPE (горные хребты, водоемы и др.).

PRODUCT: Объекты, транспортные средства, еда (но не услуги).

EVENT: Названные события – ураганы, войны, спортивные соревнования.

WORK_OF_ART: Названия книг, песен, произведений искусства.

LAW: Законы и юридические документы.

LANGUAGE: Названные языки.

DATE: Даты и временные периоды.

TIME: Временные промежутки, меньшие суток.

PERCENT: Процентные значения (включая «%»).

MONEY: Денежные суммы, включая валютные единицы.

QUANTITY: Единицы измерения (масса, расстояние и т. д.).

ORDINAL: Порядковые числительные («первый», «второй» и т. д.).

CARDINAL: Числа, не относящиеся к другим категориям.

## Часть 1. Подготовка среды
Перед началом работы необходимо установить и настроить spaCy, а также загрузить предобученную модель `en_core_web_sm`.

In [54]:

# Установка и загрузка spaCy
!pip install spacy

# Загрузка модели английского языка
!python -m spacy download en_core_web_sm

!python -m spacy download xx_ent_wiki_sm # Для мульти-языковой поддержки



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
/Users/aikei/PycharmProjects/IRIE_practice-7/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.5 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
/Users/aikei/PycharmProjects/IRIE_practice-7/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.

In [55]:

# Импорт spaCy и загрузка модели
import spacy

# Загружаем предобученную модель
nlp = spacy.load("en_core_web_sm") # Основная модель
nlp_multi = spacy.load('xx_ent_wiki_sm')  # Модель для многоязычного анализа
# Проверка работы модели
text_en = """
Artificial Intelligence (AI) is transforming the healthcare industry by improving diagnostics, treatment planning, and patient care.
In recent years, AI-driven technologies such as machine learning and natural language processing have been widely adopted in medical research and hospital management.

One of the most significant applications of AI in healthcare is medical imaging analysis. AI-powered algorithms can detect diseases such as cancer, pneumonia, and cardiovascular conditions with high accuracy, often surpassing human radiologists.
For example, Google's DeepMind has developed AI models that can diagnose eye diseases using retinal scans with a precision comparable to that of top ophthalmologists.

Another critical area is drug discovery. Traditional drug development is a long and expensive process, but AI accelerates it by analyzing vast datasets to identify potential drug candidates.
Pharmaceutical companies like Pfizer and Novartis are leveraging AI to optimize drug formulation and predict molecular interactions.

AI is also revolutionizing personalized medicine. By analyzing a patient's genetic information, lifestyle, and medical history, AI can recommend tailored treatment plans. IBM Watson Health, for instance, utilizes AI to provide oncologists with data-driven insights for cancer treatment.

Telemedicine is another sector where AI is making an impact. AI-powered chatbots and virtual assistants help patients schedule appointments, provide symptom analysis, and offer preliminary diagnoses.
During the COVID-19 pandemic, AI tools played a crucial role in tracking outbreaks and predicting virus mutations.

Despite its benefits, AI in healthcare also faces challenges. Issues like data privacy, algorithm bias, and regulatory compliance must be addressed to ensure ethical and effective implementation.

As technology advances, AI is expected to play an even greater role in preventive care, robotic surgeries, and automated patient monitoring, ultimately improving healthcare outcomes worldwide.
"""
text_ru = """
Искусственный интеллект (ИИ) трансформирует сферу здравоохранения, улучшая диагностику, планирование лечения и уход за пациентами.
В последние годы технологии на основе ИИ, такие как машинное обучение и обработка естественного языка, активно внедряются в медицинские исследования и управление больницами.

Одно из самых значительных применений ИИ в медицине — анализ медицинских изображений.
Алгоритмы, работающие на основе ИИ, способны с высокой точностью обнаруживать заболевания, такие как рак, пневмония и сердечно-сосудистые болезни, зачастую превосходя врачей-радиологов.
Например, компания DeepMind (подразделение Google) разработала ИИ-модели, которые могут диагностировать заболевания глаз по сканированию сетчатки с точностью, сравнимой с ведущими офтальмологами.

Еще одна важная область — разработка лекарств. Традиционные методы разработки новых препаратов занимают годы и требуют значительных финансовых затрат.
ИИ позволяет ускорить этот процесс, анализируя огромные массивы данных для выявления потенциальных лекарственных соединений.
Компании Pfizer и Novartis уже используют ИИ для оптимизации формул лекарств и прогнозирования взаимодействий молекул.

ИИ также революционизирует персонализированную медицину. Анализируя генетические данные пациента, его образ жизни и медицинскую историю, ИИ может рекомендовать индивидуализированные планы лечения.
IBM Watson Health, например, использует ИИ для помощи онкологам в выборе наиболее эффективных методов лечения рака.

Телемедицина — еще одна область, где ИИ оказывает значительное влияние.
Чат-боты и виртуальные ассистенты на основе ИИ помогают пациентам записываться на прием, анализировать симптомы и выдавать предварительные диагнозы.
Во время пандемии COVID-19 инструменты ИИ играли важную роль в отслеживании вспышек заболеваний и прогнозировании мутаций вируса.

Несмотря на все преимущества, использование ИИ в медицине сталкивается с рядом проблем, таких как защита персональных данных, предвзятость алгоритмов и соблюдение норм регулирования.
Для успешного внедрения необходимо решать эти вопросы.

В будущем ИИ будет играть еще более важную роль в профилактической медицине, роботизированных хирургических операциях и автоматическом мониторинге пациентов, что приведет к значительному улучшению системы здравоохранения по всему миру.
"""
text_kz = """
Жасанды интеллект (ЖИ) денсаулық сақтау саласын түбегейлі өзгертіп, диагностика, емдеу жоспары мен пациент күтімін жақсартады.
Соңғы жылдары машиналық оқыту және табиғи тілді өңдеу сияқты ЖИ технологиялары медициналық зерттеулер мен аурухана басқару жүйелеріне кеңінен енгізілуде.

ЖИ-дің ең маңызды қолдануларының бірі – медициналық суреттерді талдау.
ЖИ-мен жұмыс істейтін алгоритмдер қатерлі ісік, пневмония және жүрек-қан тамырлары ауруларын жоғары дәлдікпен анықтай алады және кейде радиолог-дәрігерлерден де асып түседі.
Мысалы, Google компаниясының DeepMind бөлімшесі көз ауруларын анықтау үшін торлы қабық сканерлеуін қолданатын ЖИ моделін жасады, оның дәлдігі ең үздік офтальмологтарға тең.

ЖИ дәрілік заттарды әзірлеу саласын да өзгертуде.
Дәстүрлі түрде дәрі жасау ұзақ әрі қымбат процесс, бірақ ЖИ үлкен деректер жиынтығын талдау арқылы жаңа дәрілерді анықтау жылдамдығын арттырады.
Pfizer және Novartis компаниялары ЖИ-ді дәрі формуласын оңтайландыру және молекулалық өзара әрекеттесуді болжау үшін қолданып келеді.

Персоналдандырылған медицина да ЖИ арқасында дамып келеді.
Науқастың генетикалық деректерін, өмір салтын және медициналық тарихын талдау арқылы ЖИ жеке емдеу әдістерін ұсынуға көмектеседі.
IBM Watson Health жүйесі, мысалы, онкологтарға ең тиімді емдеу әдістерін ұсынуда ЖИ-ді пайдаланады.

Телемедицина да ЖИ-дің үлкен ықпалын көріп отыр.
ЖИ-мен жабдықталған чат-боттар мен виртуалды көмекшілер пациенттерге кездесулерді жоспарлауға, белгілерді талдауға және алдын ала диагноз қоюға көмектеседі.
COVID-19 пандемиясы кезінде ЖИ құралдары аурулардың таралуын бақылау және вирустың мутацияларын болжауда маңызды рөл атқарды.

Дегенмен, ЖИ-дің медицинада қолданылуында мәліметтердің құпиялылығы, алгоритмдердің бейтараптығы және реттеуші нормаларды сақтау сияқты мәселелер бар.
Бұл технологияларды тиімді әрі қауіпсіз қолдану үшін осы қиындықтарды шешу қажет.

Болашақта ЖИ профилактикалық медицинада, роботтандырылған хирургияда және автоматты пациент мониторингінде одан әрі маңызды рөл атқарады, бұл бүкіл әлем бойынша денсаулық сақтау жүйесінің сапасын жақсартуға ықпал етеді.
"""

# Анализ английского текста
doc_en = nlp(text_en)
print('\nРаспознанные сущности (ENGLISH):')
for ent in doc_en.ents:
    print(f'{ent.text} ({ent.label_})')

# Анализ русского текста
doc_ru = nlp_multi(text_ru)
print('\nРаспознанные сущности (RUSSIAN):')
for ent in doc_ru.ents:
    print(f'{ent.text} ({ent.label_})')

# Анализ казахского текста
doc_kz = nlp_multi(text_kz)
print('\nРаспознанные сущности (KAZAKH):')
for ent in doc_kz.ents:
    print(f'{ent.text} ({ent.label_})')


Распознанные сущности (ENGLISH):
Artificial Intelligence (AI (ORG)
recent years (DATE)
AI (GPE)
One (CARDINAL)
AI (GPE)
Google (ORG)
DeepMind (PRODUCT)
AI (ORG)
AI (GPE)
Pfizer (PERSON)
Novartis (PERSON)
AI (GPE)
AI (GPE)
AI (GPE)
IBM Watson Health (ORG)
AI (GPE)
AI (ORG)
COVID-19 (ORG)
AI (GPE)
AI (GPE)
AI (GPE)

Распознанные сущности (RUSSIAN):
ИИ (MISC)
ИИ (MISC)
Алгоритмы (PER)
ИИ (MISC)
DeepMind (MISC)
Google (ORG)
Pfizer (ORG)
Novartis (ORG)
ИИ (MISC)
ИИ (MISC)
IBM Watson Health (MISC)
ИИ (MISC)
Телемедицина (PER)
COVID-19 (MISC)
ИИ (MISC)
ИИ (MISC)

Распознанные сущности (KAZAKH):
ЖИ (ORG)
Соңғы (PER)
ЖИ (ORG)
кеңінен енгізілуде (PER)
ЖИ-дің (PER)
– (MISC)
Google (ORG)
DeepMind (MISC)
ЖИ (PER)
ЖИ (ORG)
Дәстүрлі (PER)
ЖИ (LOC)
Pfizer (ORG)
Novartis (ORG)
ЖИ (ORG)
ЖИ (LOC)
IBM Watson Health (MISC)
Телемедицина да ЖИ-дің (PER)
COVID-19 (MISC)
ЖИ (ORG)
Дегенмен (LOC)
ЖИ-дің медицинада (PER)
Болашақта ЖИ (ORG)


Анализ результатов NER (Named Entity Recognition) в spaCy
1. Английский текст (ENGLISH)
Корректно распознанные сущности:

Google (ORG), DeepMind (PRODUCT), IBM Watson Health (ORG), COVID-19 (ORG) — эти объекты правильно классифицированы как организации или продукты.

Ошибки и проблемы:

AI (GPE) (местоположение) — AI (искусственный интеллект) не является географическим объектом.

One (CARDINAL) (числовая сущность) — ошибка классификации, так как это просто слово в тексте.

AI (ORG/GPE) повторяется многократно, но чаще всего классифицируется как местоположение, что неверно.

Вывод: модель en_core_web_sm корректно обрабатывает названия компаний и организаций, но ошибается с аббревиатурами и общеупотребительными словами.

2. Русский текст (RUSSIAN)
Корректно распознанные сущности:

DeepMind (MISC), Google (ORG), Pfizer (ORG), Novartis (ORG), IBM Watson Health (MISC), COVID-19 (MISC).

Ошибки и проблемы:

ИИ (MISC) — модель определяет "ИИ" (искусственный интеллект) как категорию MISC, что не является точной категорией.

Телемедицина (PER) — неправильно классифицировано как персона.

Алгоритмы (PER) — алгоритмы не являются именованной сущностью, ошибка распознавания.

Вывод: модель xx_ent_wiki_sm для русского языка работает хуже, чем для английского, так как путает общеупотребительные термины с именами и организациями.

3. Казахский текст (KAZAKH)
Корректно распознанные сущности:

Google (ORG), DeepMind (MISC), Pfizer (ORG), Novartis (ORG), IBM Watson Health (MISC), COVID-19 (MISC).

Ошибки и проблемы:

ЖИ (ORG) — "ЖИ" (жасанды интеллект) определяется как организация, хотя это технология.

Соңғы (PER) — неправильно классифицировано как персона, хотя это слово "Последний".

Телемедицина да ЖИ-дің (PER) — неправильно распознано как персона, хотя это термин "телемедицина".

Болашақта ЖИ (ORG) — неправильно классифицировано как организация.

Вывод: казахский текст распознается хуже, чем английский и русский. Ошибки связаны с тем, что модель xx_ent_wiki_sm плохо адаптирована к казахскому языку.

Выводы и рекомендации
Ошибка распознавания аббревиатур (AI, ИИ, ЖИ) — следует добавлять пользовательские правила через EntityRuler.

Ошибка классификации общеупотребительных слов (One, Соңғы, Телемедицина) — можно улучшить с помощью расширенной модели или дообучения.

Модель xx_ent_wiki_sm хуже работает с русским и казахским языком — возможно, стоит использовать ru_core_news_sm для русского.

Следующие шаги:

Добавить правила в EntityRuler для исправления ошибок.

Попробовать другую модель для русского (ru_core_news_sm).

Создать кастомные правила для казахского языка, так как стандартная модель плохо справляется.

## Часть 2. Добавление пользовательских правил с EntityRuler
EntityRuler позволяет добавлять пользовательские правила для распознавания сущностей.

In [56]:
import spacy
from spacy.pipeline import EntityRuler

# Загрузка предобученной модели
nlp = spacy.load("en_core_web_sm")
nlp_multi = spacy.load('xx_ent_wiki_sm')  # Для русского и казахского текста

# Создание объекта EntityRuler
ruler = nlp.add_pipe("entity_ruler", before="ner")

# Определение пользовательских правил для исправления ошибок NER
patterns = [
    {"label": "EVENT", "pattern": "Expo 2017"},

    # Географические объекты (исправлено на актуальные города)
    {"label": "GPE", "pattern": "Almaty"},
    {"label": "GPE", "pattern": "Astana"},

    # Исправление ошибочного определения "AI" как GPE
    {"label": "TECH", "pattern": "AI"},
    {"label": "TECH", "pattern": "ИИ"},
    {"label": "TECH", "pattern": "ЖИ"},

    # Организации
    {"label": "ORG", "pattern": [{"LOWER": "satbayev"}, {"LOWER": "university"}]},
    {"label": "ORG", "pattern": "OpenAI"},
    {"label": "ORG", "pattern": "Google"},
    {"label": "ORG", "pattern": "IBM Watson Health"},
    {"label": "ORG", "pattern": "Pfizer"},
    {"label": "ORG", "pattern": "Novartis"},
    {"label": "ORG", "pattern": "DeepMind"},
    {"label": "ORG", "pattern": "COVID-19"},  # Добавлен в список организаций

    # Исправление неправильного определения "Телемедицина" как PERSON
    {"label": "MEDICAL_FIELD", "pattern": "Телемедицина"},

    # Денежные суммы
    {"label": "MONEY", "pattern": [{"TEXT": "$"}, {"IS_DIGIT": True}, {"TEXT": "billion"}]},

    # Даты
    {"label": "DATE", "pattern": [{"TEXT": {"REGEX": r"\d{4}"}}]},

    # Числовые сущности
    {"label": "CARDINAL", "pattern": [{"LOWER": "one"}]},  # Исправление ошибки с "One"

    # Исправление ошибок в казахском языке
    {"label": "GPE", "pattern": "Қазақстан"},  # Казахстан
    {"label": "MISC", "pattern": "Соңғы"},  # Исправлено, не является персоной
    {"label": "MEDICAL_FIELD", "pattern": "Телемедицина да ЖИ-дің"}  # Исправлено
]

# Добавление правил в EntityRuler
ruler.add_patterns(patterns)


## Часть 3. Работа с шаблонами
В этой части рассмотрим различные подходы к созданию шаблонов для EntityRuler.

In [57]:
# Примеры различных шаблонов, исправляющие предыдущие ошибки
patterns = [
    # Исправление ошибки классификации имен
    {"label": "PERSON", "pattern": "Elon Musk"},

    # Исправление ошибок с AI, ИИ, ЖИ
    {"label": "TECH", "pattern": "AI"},
    {"label": "TECH", "pattern": "ИИ"},
    {"label": "TECH", "pattern": "ЖИ"},

    # Исправление неправильного определения "One" как числовой сущности
    {"label": "CARDINAL", "pattern": "One"},

    # Улучшение определения продуктов
    {"label": "PRODUCT", "pattern": [{"LOWER": "iphone"}, {"IS_DIGIT": True}]},
    {"label": "PRODUCT", "pattern": "DeepMind"},

    # Исправление ошибки с COVID-19, теперь определяется как ORG
    {"label": "ORG", "pattern": "COVID-19"},

    # Исправление ошибок в русском языке (телемедицина не является персоной)
    {"label": "MEDICAL_FIELD", "pattern": "Телемедицина"},
    {"label": "MEDICAL_FIELD", "pattern": "Телемедицина да ЖИ-дің"},

    # Исправление географических объектов (Nur-Sultan убран, Astana добавлен)
    {"label": "GPE", "pattern": "Almaty"},
    {"label": "GPE", "pattern": "Astana"},
    {"label": "GPE", "pattern": "Қазақстан"},

    # Улучшение определения дат
    {"label": "DATE", "pattern": [{"TEXT": {"REGEX": r"\d{4}"}}]},

    # Улучшение распознавания денежных сумм
    {"label": "MONEY", "pattern": [{"TEXT": "$"}, {"IS_DIGIT": True}, {"TEXT": "billion"}]},
]

# Добавление новых шаблонов в EntityRuler
ruler.add_patterns(patterns)


## Часть 4. Практическое задание
### Задача 1: Добавление правил для городов, событий и организаций

In [58]:
# Добавляем правила для городов, событий, организаций и исправляем ошибки
patterns = [
    # Города
    {"label": "GPE", "pattern": "Almaty"},
    {"label": "GPE", "pattern": "Astana"},
    {"label": "GPE", "pattern": "Қазақстан"},  # Казахстан теперь правильно распознается

    # События
    {"label": "EVENT", "pattern": "Expo 2017"},

    # Организации
    {"label": "ORG", "pattern": "OpenAI"},
    {"label": "ORG", "pattern": "Google"},
    {"label": "ORG", "pattern": "IBM Watson Health"},
    {"label": "ORG", "pattern": "Pfizer"},
    {"label": "ORG", "pattern": "Novartis"},
    {"label": "ORG", "pattern": "DeepMind"},
    {"label": "ORG", "pattern": "COVID-19"},  # Исправлено: теперь определяется как ORG

    # Технологии (исправлено: AI, ИИ, ЖИ теперь не будут ошибочно классифицироваться)
    {"label": "TECH", "pattern": "AI"},
    {"label": "TECH", "pattern": "ИИ"},
    {"label": "TECH", "pattern": "ЖИ"},

    # Медицинская сфера (исправление: Телемедицина теперь не распознается как персона)
    {"label": "MEDICAL_FIELD", "pattern": "Телемедицина"},
    {"label": "MEDICAL_FIELD", "pattern": "Телемедицина да ЖИ-дің"},

    # Денежные суммы и даты (улучшено распознавание чисел)
    {"label": "MONEY", "pattern": [{"TEXT": "$"}, {"IS_DIGIT": True}, {"TEXT": "billion"}]},
    {"label": "DATE", "pattern": [{"TEXT": {"REGEX": r"\d{4}"}}]},
]

# Добавление новых шаблонов в EntityRuler
ruler.add_patterns(patterns)

### Задача 2: Создание шаблонов для денежных сумм и лет

In [59]:
# Добавляем шаблоны для денежных сумм и годов
patterns = [
    # Денежные суммы (улучшено: добавлены другие валюты и вариации написания)
    {"label": "MONEY", "pattern": [{"TEXT": "$"}, {"IS_DIGIT": True}, {"TEXT": "billion"}]},
    {"label": "MONEY", "pattern": [{"TEXT": "$"}, {"IS_DIGIT": True}, {"TEXT": "million"}]},
    {"label": "MONEY", "pattern": [{"IS_DIGIT": True}, {"TEXT": "tenge"}]},
    {"label": "MONEY", "pattern": [{"TEXT": "₸"}, {"IS_DIGIT": True}]},

    # Даты (улучшено: поддержка диапазонов, месяцев и сокращений)
    {"label": "DATE", "pattern": [{"TEXT": {"REGEX": r"\d{4}"}}]},  # Годы (например, 2024)
    {"label": "DATE", "pattern": [{"TEXT": {"REGEX": r"\d{1,2}"}}, {"LOWER": {"IN": ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]}}, {"TEXT": {"REGEX": r"\d{4}"}}]},  # Даты вида "12 March 2024"
    {"label": "DATE", "pattern": [{"TEXT": {"REGEX": r"\d{1,2}"}}, {"LOWER": {"IN": ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]}}, {"TEXT": {"REGEX": r"\d{4}"}}]},  # Сокращенные названия месяцев "12 Mar 2024"
    {"label": "DATE", "pattern": [{"TEXT": {"REGEX": r"\d{4}-\d{4}"}}]},  # Годовые диапазоны (например, 2010-2020)
]

# Добавление новых шаблонов в EntityRuler
ruler.add_patterns(patterns)

### Задача 3: Тестирование правил на тексте

In [60]:
# Тестируем правила
text = """
In 2023, OpenAI announced new AI research projects.
Google invested $10 billion in cloud computing in Almaty and Astana.
In March 2024, Apple launched a new iPhone 15.
Kazakhstan allocated ₸500 million for AI development.
Between 2015-2020, Tesla made major advancements in self-driving technology.
"""

doc = nlp(text)

print("Распознанные сущности:")
for ent in doc.ents:
    print(f"{ent.text} ({ent.label_})")


Распознанные сущности:
2023 (DATE)
OpenAI (ORG)
AI (TECH)
Google (ORG)
$10 billion (MONEY)
Almaty (GPE)
Astana (GPE)
2024 (DATE)
Apple (ORG)
iPhone 15 (PRODUCT)
Kazakhstan (GPE)
₸500 (MONEY)
AI (TECH)
2015 (DATE)
2020 (DATE)
Tesla (ORG)


## Часть 5. Визуализация результата
Используем `displacy` для визуализации именованных сущностей.

In [61]:
from spacy import displacy

# Визуализация текста на английском
print("\nВизуализация текста (ENGLISH):")
doc_en = nlp(text_en)  # Применяем NLP модель
displacy.render(doc_en, style="ent", jupyter=True)

# Визуализация текста на русском
print("\nВизуализация текста (RUSSIAN):")
doc_ru = nlp_multi(text_ru)  # Используем многоязычную модель
displacy.render(doc_ru, style="ent", jupyter=True)

# Визуализация текста на казахском
print("\nВизуализация текста (KAZAKH):")
doc_kz = nlp_multi(text_kz)  # Используем многоязычную модель
displacy.render(doc_kz, style="ent", jupyter=True)


Визуализация текста (ENGLISH):



Визуализация текста (RUSSIAN):



Визуализация текста (KAZAKH):


## Часть 6. Отчет по заданию

В ходе выполнения задания были разработаны и успешно интегрированы пользовательские правила для Named Entity Recognition (NER) в библиотеке spaCy. Они протестированы на текстах на английском, русском и казахском языках, что позволило оценить их точность и выявить возможные улучшения.

Итоги работы:
Настроены и добавлены правила для ключевых сущностей (города, организации, события, денежные суммы, даты и т. д.).

Проведена проверка работы NER на текстах, включающих реальные данные.

Произведен анализ ошибок распознавания, выявлены неточности в классификации отдельных сущностей (например, аббревиатуры ИИ/ЖИ/AI, терминов "телемедицина", "алгоритмы").

Реализована визуализация извлеченных сущностей, что позволило наглядно оценить работу модели.

Применение:
Настроенная система расширяет стандартные возможности spaCy NER, адаптируя ее для конкретных задач, таких как:

Обработка текстов, связанных с Казахстаном (города, академические учреждения, организации).

Финансовая аналитика (распознавание денежных сумм, экономических данных).

Научные и технологические тексты (термины, компании, технологии).

Использованные методы могут быть доработаны для дальнейшего повышения точности NER, включая добавление новых правил и обучение кастомной модели.





